## Contact
#### [linkedin](https://www.linkedin.com/in/justin-isinguzo-464bb083/)  &emsp;  [twitter](https://twitter.com/barrizm)  &emsp;[facebook](https://facebook.com/barrizmjay)

<h1><center>Linear programming(Assignment Problem)</center></h1> <br>  <h2><center>Case study: MYB Survey Project</center></h2>

## Problem Formulation.

Surveys are to be carried out in five states in different parts of the country. these jobs will be carried out by four surveyors in MYB and must be completed on or before twenty one(21) days. The time for the surveyors to complete the jobs are shown in the table. The values in the cells assume that each job is completed by a single surveyor; however, jobs can be shared, with completion times being determined proportionally If no entry exists in a particular cell, it means that the corresponding job cannot be performed by the corresponding surveyors.


<table style="width:60%">
  <tr>
    <th></th>
    <th>Abia</th>
    <th>Akwa Ibom</th>
    <th>Edo</th>
    <th>Enugu</th>
    <th>Rivers</th>
     </tr>
        
  <tr>
    <td>Team RPM</td>
    <td></td>
    <td>12</td>
    <td></td>
    <td></td>
    <td>14</td>
    </tr>

  <tr>
    <td>Peter</td>
    <td>12</td>
    <td></td>
    <td>4</td>
    <td></td>
    <td></td>
    </tr>

  <tr>
    <td>Douglas</td>
    <td>16</td>
    <td></td>
    <td></td>
    <td>7</td>
    <td></td>
    </tr>
    
  <tr>
    <td>Andrew</td>
    <td></td>
    <td>18</td>
    <td></td>
    <td>9</td>
    <td>20</td>
    </tr>
   <tr>
    <th>Meters</th>
    <td>84906</td>
    <td>94569</td>
    <td>30857</td>
    <td>44623</td>
    <td>105353</td>
</tr>
</table>

This model seeks to optimize the time required to complete this project by determining the best assignment of surveyors to jobs so as to meet up to the set target provided by HAUWIE on the contract. 

* Objective: Minimize the time that will be taken to complete this survey
* LP Form: Minimization(pyomo)
* Decision Variables: Twenty(20) Variables .
* Constraints: All the jobs must be completed and the total number of days for completion must not exceed twenty one days

<div class="alert alert-block alert-info">
<b>Why Concrete model</b><br>
Concrete model is prefered in this case because the variables are not large hence inputs can be made directly to solve the problem.
</div>


In [139]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

To convert the table to dictionary for iteration, that is converting it to a matrix   

In [127]:
surveyors = {'Team_RPM', 'Peter', 'Douglas', 'Andrew'}

tasks = tasks = ('Abia','Akwa Ibom','Edo', 'Enugu', 'Rivers')
c = {
    ('Team_RPM',  'Akwa Ibom'):  12,
    ('Team_RPM',  'Rivers'):  14,
    ('Peter', 'Abia'): 12,
    ('Peter',  'Edo'): 4,
    ('Douglas',  'Abia'): 16,
    ('Douglas',  'Enugu'): 7,
    ('Andrew',  'Akwa Ibom'): 18,
    ('Andrew',  'Rivers'): 20,
    ('Andrew', 'Enugu'):  9,
}

max_hours = 18


In [128]:
model = pe.ConcreteModel()

In [129]:
model.surveyors = pe.Set(initialize=surveyors)
model.tasks = pe.Set(initialize=tasks)

    data source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo


In [130]:
model.c = pe.Param(model.surveyors, model.tasks, initialize=c, default=1000)
model.max_hours = pe.Param(initialize=max_hours)

In [131]:
model.x = pe.Var(model.surveyors, model.tasks, domain=pe.Reals, bounds=(0, 1))

In [132]:
expr = sum(model.c[w, t] * model.x[w, t]
           for w in model.surveyors for t in model.tasks)
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

In [133]:
model.tasks_done = pe.ConstraintList()
for t in model.tasks:
    lhs = sum(model.x[w, t] for w in model.surveyors)
    rhs = 1
    model.tasks_done.add(lhs == rhs)

In [134]:
model.hour_limit = pe.ConstraintList()
for w in model.surveyors:
    lhs = sum(model.c[w, t] * model.x[w, t] for t in model.tasks)
    rhs = model.max_hours
    model.hour_limit.add(lhs <= rhs)

In [135]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True )

Restricted license - for non-production use only - expires 2023-10-25
Read LP format model from file C:\Users\BARRIZM\AppData\Local\Temp\tmpth859j3k.pyomo.lp
Reading time = 0.05 seconds
x21: 10 rows, 21 columns, 41 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 10 rows, 21 columns and 41 nonzeros
Model fingerprint: 0x1b9206e7
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [4e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.000000e+00   0.000000e+00      0s
       6    5.2428571e+01   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.242857143e+01


In [136]:
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(model.x, names=['Surveyor', 'State']))
df['x'] = [pe.value(model.x[key]) for key in df.index]
df['c'] = [model.c[key] for key in df.index]

In [137]:
(df['c'] * df['x']).unstack('State')

State,Abia,Akwa Ibom,Edo,Enugu,Rivers
Surveyor,,,,,
Andrew,0.0,0.0,0.0,0.0,11.428571
Douglas,0.0,0.0,0.0,7.0,0.000000
Peter,12.0,0.0,4.0,0.0,0.000000
Team_RPM,0.0,12.0,0.0,0.0,6.000000


## Interpretation

The above result means that:

* Andrew would work in Rivers state for 12 days covering all availabe 60km in the state.
* Douglas wolud work in Enugu for 7 days covering 100% of the job available
* Peter will handle all jobs available in Abia state and Edo state covering a total of 16 days
* Team_RPM will cover the total available jobs in Akwa Ibom after wards head to Rivers for a six days stay to complete the 43% job availabe for them. 

In [138]:
(df['c'] * df['x']).groupby('Surveyor').sum().to_frame()

,0
Surveyor,
Andrew,11.428571
Douglas,7.000000
Peter,16.000000
Team_RPM,18.000000


 <div class="alert alert-block alert-info">
<b> Number of days each surveyor is expected to spend on the project</b><br>

</div>

In [19]:
df['x'].groupby('t').sum().to_frame().T

t,Abia,Akwa Ibom,Edo,Enugu,Rivers
x,1.0,1.0,1.0,1.0,1.0


 <div class="alert alert-block alert-info">
<b> To Check if the jobs were completed or not</b><br>
    The number '1' means the job schedule is complete

</div>